In [ ]:
import glob
import sep
import astropy.units as u
import astropy.coordinates as coord
import matplotlib.pyplot as plt
import numpy as np
import astroalign as aa
from scipy import odr
import matplotlib.colors as mcolors
from astropy.io import fits
from astroquery.ipac.irsa import Irsa
from astroquery.sdss import SDSS
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.coordinates import SkyCoord
from matplotlib.colors import LogNorm, TABLEAU_COLORS, CSS4_COLORS
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
from regions import CirclePixelRegion, PixCoord
from photutils import centroids

#imports
#FOR ASTROQUERRY: If you encounter an error related to "fields" in "SDSS.querry_crossid", try installing the newest development version of astroquerry rather than the default. There is a bug in the older official release version.

In [ ]:
class Night: #Each observing night is initialized as a Night object
    def __init__(self, file_path, night_number):
        #each attribute is declared here first, even if value is assigned later
        self.path = file_path #file path of obs night
        self.obs_night = night_number #each night is assigned a number, the first night is night 0
        self.image_data = None #the image data for the night
        self.headers = None #the night's headers
        self.wcs = None #world coordinate system transformation matrix
        self.readout_noise = None #detector readout noise
        self.aligned = None #aligned image data
        self.template = None #template frame
        self.references = None #reference sources
        self.obs_filter = None #observation filter
        self.no_obs_images = None #number of images in night


    def initialize_frames(self):
        science_dir = sorted(glob.glob(self.path + '/*')) #pulls data files using glob
        hdus = [fits.open(image) for image in science_dir] #opens each fits image
        self.image_data = [image[1].data for image in hdus] #pulls image data from file
        self.headers = [image[1].header for image in hdus] #pulls header data from file.
        self.wcs = WCS(self.headers[0]) #gets wcs transformation, each night should only have one unique transformation
        self.readout_noise = self.headers[0]['RDNOISE'] #pulls readout noise from header. Readout noise is detector based and should be the same across nights taken with same equipment but do this just in case.

        try: #try to align images for best results but might be able to get away with not.
            self.aligned_images = [aa.register(image, self.image_data[0])[0] for image in self.image_data[0:]]
        except:
            self.aligned_images = self.image_data

        self.template = np.median(self.aligned_images, axis = 0) #night template
        background = sep.Background(self.template) #sep background subtraction for source extraction
        self.references = sep.extract(self.template - background.back(),  background.globalrms*3, minarea =25, segmentation_map=False) #finds sources based on given parameters

        self.obs_filter = self.headers[0]['filter'][0] #observation filter.
        self.no_obs_images = len(self.aligned_images) #number of images in night.

    def get_info(self): #function to grab night info, useful for debugging.
        print(f"path: {self.path}, night {self.obs_night}, n_frames: {len(self.image_data)}, n_aligned: {len(self.aligned_images)}, wcs: {self.wcs}, n_ref: {len(self.references)}, filter: {self.obs_filter}")

In [ ]:
class Source: #initialize source object
    def __init__(self, source, count, WCS):
        self.position = pixel_to_skycoord(source['x'], source['y'], wcs= WCS).transform_to('icrs') #since pixel locations are inconsistent, store position as RA/DEC
        self.radius = (source['xmax'] - source['xmin'])/2 #source radius (size) provided by SEP
        self.source_id = count #identifying number
        self.is_reference = None #if star is reference
        self.ref_mag = None #SDSS magnitude, if available
        self.ref_mag_err = None #reference mag error
        self.inst_mags = [] #instrumnetal (our) magnitudes
        self.inst_mag_errs = [] #instrumental mag errors
        self.calibrated_mags = [] #calibrated magnitudes
        self.flagged = False #bad source flag. Will be flipped true if a source is not present in all observing nights or has negative aperture sum

    def query_source(self): #querry a source through the sdss database
        #we want the search to return ra, dec, mags, and mag error. region = False, returns first result of search.
        search = SDSS.query_crossid(self.position, fields = ['ra', 'dec', f'psfMag_{Nights[0].obs_filter}', f'psfMagErr_{Nights[0].obs_filter}'], radius = 15 * u.arcsec, region = False)
        if search:
            if search['type'] == "STAR": #want to make sure reference objects are Stars
                self.is_reference = True #lets us know star is a reference star
                self.ref_mag = search[f'psfMag_{Nights[0].obs_filter}'] #fill in mag and error fields:
                self.ref_mag_err = search[f'psfMagErr_{Nights[0].obs_filter}']

    def boundary_check(self, night): #checks if a star is within frame for a given night.
        source_xy = SkyCoord.to_pixel(self.position, wcs= night.wcs)
        if (night.headers[0]['NAXIS1'] - source_xy[0]) < 0 or source_xy[0] < 0 or (night.headers[0]['NAXIS2'] - source_xy[1]) < 0 or source_xy[1] < 0:
            self.flagged = True #if star is out of bounds, flags star as bad


    def aperture_photometry(self, img, nght): #does aperture photometry

        coords = SkyCoord.to_pixel(self.position, wcs = nght.wcs) #gets pixel values of source from RA DEC
        pcoords = PixCoord(coords[0], coords[1]) #another coord object needed for Regions

        radius_i = self.radius #inner aperture radius
        radius_o_0 = radius_i + 5 #inner annulus radius
        radius_o_1 = radius_o_0 + 5 #outer annulus radius

        source_circle = CirclePixelRegion(pcoords, radius_i).to_mask() #makes region of source shape
        source_aperture = source_circle.cutout(img) #gets data of source

        background_annulus = CircularAnnulus(coords, radius_o_0, radius_o_1) #makes annulus for background subtraction
        background_mean = ApertureStats(img, background_annulus).mean #takes mean of background annulus


        source_flux_pix = source_aperture-(source_circle*background_mean) #pixel wise background subtraction
        source_flux_total = np.sum(source_flux_pix) #total flux


        readout_sum_square = np.sum(source_circle*np.float64(nght.readout_noise**2)) #applies square readout noise to source array shape, then adds. Gives sum of square readout noise over back subtracted source.

        delta_n = (readout_sum_square + source_flux_total + (((radius_i**2)/((radius_o_1**2)-(radius_o_0**2)))**2)*(readout_sum_square + aperture_photometry(img, background_annulus)['aperture_sum'][0]))**(1/2) #term needed for magnitude error calculation

        if source_flux_total < 0:
            print(self.source_id)
            self.flagged = True #flags source if aperture sum turns out to be negative

        else:
            instrumental_mag = -2.5*np.log10(source_flux_total) #magnitude
            instrumental_mag_error = 2.5*np.log10(np.e)*abs(delta_n/source_flux_total) #magntiude error
            self.inst_mags.append(instrumental_mag)
            self.inst_mag_errs.append(instrumental_mag_error)

    def add_calibrated_mag(self, mag):
        self.calibrated_mags.append(mag) #adds calibrated mag. For some reason, math comes out unexpectedly if calibration takes place in class function.


    def get_info(self): #prints out source info.
        print(f"ra_dec: {self.position}, rad: {self.radius}, ref_status: {self.is_reference}, ref_mag: {self.ref_mag}, inst_mag_avg:{np.mean(self.inst_mags)}, cal_mag_avg: {np.mean(self.calibrated_mags)}, flagged: {self.flagged}, ID: {self.source_id}")


In [ ]:
def lin_model(p, x): #define a standard linear model for ODR fitting. Part of calibration.
    return p[0] * x + p[1]

In [ ]:
primary_dir = sorted(glob.glob("/Users/lucaangeleri/Documents/LCO/SN2023ixf_all/SN2023ixf_r/*")) #main file directory; directory containing folders for each night.
Nights = [Night(directory, dir_number) for dir_number, directory in enumerate(primary_dir)] #initializes each night as a Night object
for night in Nights:
    night.initialize_frames() #see initialize_frames() in Night class definitions

In [ ]:
Sources = [Source(source, count, Nights[0].wcs) for count, source in enumerate(Nights[0].references)] #initializes sources based off first night's list. This ensures proper source traking
for source in Sources:
    if source.flagged == False:
        source.query_source() #see query_source() in source class definitions

In [ ]:
night_array = [] #this is to help organize plotting later.
mag_thresh = 16.5 #magnitude threshold for calibrating sources.
for night in Nights: #for each night, iterates through every source for each image.
    for image in night.aligned_images:
        for source in Sources:
            source.boundary_check(night) #see boundary_check() in source class definition
            if source.flagged == False:
                source.aperture_photometry(image, night)  #see aperture_photometry() in source class definition
        night_array.append(night.obs_night)


In [ ]:
slopes = []
zeros = []
for night in Nights: #does orthogonal distance regression to fit a calibration line. Takes average mags across each night for a total of 10 calibration plots.
    index_low = len(night.image_data)*night.obs_night
    index_high = len(night.image_data)*(1+night.obs_night)
    x = [np.mean(source.inst_mags[index_low:index_high]) for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    y = [source.ref_mag[0] for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    inst_mag_errs = [np.mean(source.inst_mag_errs[index_low:index_high]) for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    sky_mag_errs = [source.ref_mag_err[0] for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    linear = odr.Model(lin_model)
    calibration_data = odr.Data(x, y, we=inst_mag_errs, wd = sky_mag_errs)
    fit_params = odr.ODR(calibration_data, linear, beta0=[1.0, 23.5]).run() #beta0 is initial guesses
    slopes.append(fit_params.beta[0])
    zeros.append(fit_params.beta[1])

print(len(slopes))

In [ ]:
slopes_scaled = np.array(slopes)[np.array(night_array)]
zeros_scaled = np.array(zeros)[np.array(night_array)]
for source in Sources:
    if source.flagged != True:
        for i in range(0, len(slopes_scaled)):
            #print(i, slopes_scaled[i], zeros_scaled[i])
            mag = (source.inst_mags[i]*slopes_scaled[i] + zeros_scaled[i])
            source.add_calibrated_mag(mag)

In [ ]:
color_arr = np.array(sorted(CSS4_COLORS, key=lambda c: tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(c)))))
l = np.arange(0, len(slopes_scaled))

median_curves = []
for night in Nights:
    median_mags = []
    index_l = len(night.image_data)*night.obs_night
    index_h = len(night.image_data)*(1+night.obs_night)
    for source in Sources:
        if source.flagged != True:
            median_mags.append(source.calibrated_mags[index_l:index_h])
    median_curve = np.median(median_mags, axis = 0)/np.median(median_mags)
    median_curves.append(median_curve)
med_curve = np.concatenate(median_curves)
print(len(med_curve))
plt.scatter(l, med_curve, c = color_arr[93::5][np.array(night_array)])
plt.gca().invert_yaxis()
plt.show()

In [ ]:
for source in Sources:
    if source.flagged != True:
        print(source.get_info())
        r = np.arange(0, len(source.calibrated_mags))
        for i, value in enumerate(source.calibrated_mags):
            plt.errorbar(i, value/med_curve[i], yerr=source.inst_mag_errs[i], linestyle = 'none', marker = 'o', c = color_arr[93::5][np.array(night_array)][i] )
        avg_mag =  np.mean(source.calibrated_mags)
        avg_mag_str = "%.3f" % avg_mag
        plt.plot(r, np.ones(len(r))*avg_mag, linestyle = '--', color = "g", label = f"TRIPP{Nights[0].obs_filter} AVG Mag: {avg_mag_str}")
        if source.is_reference:
            plt.plot(r, np.ones(len(r))*source.ref_mag, linestyle = '--', color = "r", label = f"SDSS-{Nights[0].obs_filter} Ref Mag: {source.ref_mag}")
            print(source.ref_mag-avg_mag)

        plt.gca().invert_yaxis()
        plt.legend()
        #plt.savefig("/Users/lucaangeleri/Desktop/test/plot{}".format(source.source_id), format = 'png', dpi = 500,  bbox_inches="tight")
        plt.show()


In [ ]:
for night in Nights:
    index_low = len(night.image_data)*night.obs_night
    index_high = len(night.image_data)*(1+night.obs_night)
    x = [np.mean(source.calibrated_mags[index_low:index_high]) for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    y = [source.ref_mag[0] for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    inst_mag_errs = [np.mean(source.inst_mag_errs[index_low:index_high]) for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    sky_mag_errs = [source.ref_mag_err[0] for source in Sources if source.is_reference == True and source.ref_mag < mag_thresh and source.flagged == False]
    print(len(x),  index_low, index_high, night.obs_night)
    #p = np.polyfit(x, y, deg = 1)
    linear = odr.Model(lin_model)
    calibration_data = odr.Data(x, y, we=inst_mag_errs, wd = sky_mag_errs)
    fit_params = odr.ODR(calibration_data, linear, beta0=[1.0, 0]).run()
    #r = np.arange(0, len(x))
    r = np.arange(5, 18)
    plt.scatter(x, y, c = 'r')
    plt.plot(r, r*fit_params.beta[0] + fit_params.beta[1], c = 'b' )
    plt.title(f"fit: m: {fit_params.beta[0]}, b: {fit_params.beta[1]}")
    plt.show()